# Bing Search Grounding Agent

In [5]:
import os
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool

# Load environment variables from .env file
load_dotenv()
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]
bing_connection_name = os.environ["BING_CONNECTION_NAME"]

# Create an Azure AI Client from the connection string
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=project_connection_string,
)

with project_client:
    # Retrieve the Bing connection using the connection name
    bing_connection = project_client.connections.get(connection_name=bing_connection_name)
    conn_id = bing_connection.id
    print(f"Bing Connection ID: {conn_id}")
    
    # Initialize the Bing Grounding Tool with the connection ID
    bing = BingGroundingTool(connection_id=conn_id)
    
    # Create the agent using the Bing Grounding tool
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="my-assistant",
        instructions="You are a helpful assistant.",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    print(f"Created agent, ID: {agent.id}")
    
    # Create a conversation thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")
    
    # Create a message in the thread (for example, asking for top news)
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What are the top Artificial Intelligence news today?",
    )
    print(f"Created message, ID: {message.id}")
    
    # Process the agent run (the agent will use Bing grounding to answer)
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id,
        agent_id=agent.id
    )
    print(f"Run finished with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
    # Retrieve run steps for further details (e.g., Bing search query details)
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    print(f"Run steps: {run_steps}")
    
    # Retrieve and print all messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    
    last_msg_from_assistant = None
    if "data" in messages and isinstance(messages["data"], list):
        for m in reversed(messages["data"]):
            if m.get("role") == "assistant":
                last_msg_from_assistant = m
                break

    # Print the last assistant message (text content)
    if last_msg_from_assistant:
        for content_item in last_msg_from_assistant.get("content", []):
            if content_item["type"] == "text":
                text_value = content_item["text"]["value"]
                print(f"Assistant: {text_value}")
                
    else:
        print("No messages found from the assistant.")
        
    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)


Bing Connection ID: /subscriptions/d4cd7aa5-7232-42b5-8538-8a931412d74f/resourceGroups/agent-rg-713/providers/Microsoft.MachineLearningServices/workspaces/florianfollonier7-0437-regular/connections/binggrounding713
Created agent, ID: asst_TicLK8VmuUTLUAYStYHRpJPd
Created thread, ID: thread_gQCP7NxVR30JdthhBsHI4MBJ
Created message, ID: msg_rUYGq6cs8zcJGK3D1WDiFnbj
Run finished with status: RunStatus.COMPLETED
Run steps: {'object': 'list', 'data': [{'id': 'step_gTa8vV6aoHjIDA8C30lNjo8L', 'object': 'thread.run.step', 'created_at': 1743694239, 'run_id': 'run_733sxb4CDpmcoPnSIOjC6cQM', 'assistant_id': 'asst_TicLK8VmuUTLUAYStYHRpJPd', 'thread_id': 'thread_gQCP7NxVR30JdthhBsHI4MBJ', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1743694240, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_QuLIF090atyuIYfiIAg7Mu8y'}}, 'usage': {'prompt_tokens': 2472, 'complet